# EE194 Lab 0: Chisel - Part 5: Inheritance

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE" or `YOUR ACTION NEEDED HERE`.

If you see `???` right below `YOUR CODE HERE`, make sure to remove that after you have implemented your solution (and before you run the code block).

### Import the necessary Chisel dependencies. 
> There will be cells like these in every lab. Make sure you run them before proceeding to bring the Chisel Library into the Jupyter Notebook scope!

In [ ]:
interp.configureCompiler(_.settings.processArguments(List("-Wconf:cat=deprecation:s"), true))
interp.load.module(os.Path(s"${System.getProperty("user.dir")}/resource/chisel_deps.sc"))

In [ ]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test

## Summary of Methods of Reuse in Chisel
* **Parameterized hardware generators** - hopefully sufficiently flexible to be used in more places
* **Composable/customizable Bundles** - can reduce effort defining interfaces, can abstract away complex IO interfaces
* **Inheritance** (this lab) - Similar modules can share functionality
    * Note: Chisel itself is implemented using inheritance (ie: when you `extend Module`)

## Scala `abstract class`
* Sometimes we don't want to provide implementations of the inherited things, and we want to leave this up to the classes that inherits from the abstract class.
* Can't instantiate an abstract class, must inherit from it
* For multiple inheritance, will need to consider `trait` (later below)

Here is a Scala example, and then a Chisel example:

In [ ]:
abstract class Parent(name: String) {
    val phrase: String // instance variable, each class that inherits from this abstract class will have their own values.
    
    def greet() { println(s"$phrase $name") } 
}

class InEnglish(name: String) extends Parent(name) {
    val phrase = "hello"
}

val e = new InEnglish("Kate")
e.greet

class InFrench(name: String) extends Parent(name) {
    val phrase = "bonjour"
}

val f = new InFrench("Antoine")
f.greet

class InSpanish(name: String) extends Parent(name) {
    val phrase = "hola" // this is still an instance variable

    // we can override the greet method, unless `greet()` is defined as a `final def greet()`
    override def greet() { 
        println(s"¡$phrase, $name!") 
    }
}

val s = new InSpanish("Carlos")
s.greet() 

In [ ]:
abstract class UnaryOperatorModule(width: Int) extends Module {
    def op(x: UInt): UInt // to be implemented by each class that inherits from this abstract class
    val io = IO(new Bundle {
        val in = Input(UInt(width.W))
        val out = Output(UInt(width.W))
    })
    io.out := op(io.in) // calls whatever `op` function each class implemented on `io.in`
}

class PassThruMod(width: Int) extends UnaryOperatorModule(width) {
    def op(x: UInt) = x
}

class NegMod(width: Int) extends UnaryOperatorModule(width) {
    def op(x: UInt) = ~x
}

printVerilog(new PassThruMod(8))
printVerilog(new NegMod(8))

## Abstract Class + Case Class pattern matching
> Fill in the body of `ProcessRequest` using pattern matching to translate a requested `MealItem` into a more descriptive string. The meal items are case classes inheriting from MealItem. You can infer what the strings should be by looking at the following testcases.

For Scala syntax help on pattern matching, see: https://docs.scala-lang.org/tour/pattern-matching.html

In [ ]:
abstract class MealItem

case class Drink(variety: String) extends MealItem

case class Burger(withCheese: Boolean) extends MealItem

case object Fries extends MealItem

def ProcessRequest(request: MealItem): String = {
    request match {
        // YOUR CODE HERE
        ???
    }
}

def ProcessOrder(order: Seq[MealItem]) {
    order foreach { item => println(ProcessRequest(item)) }
}

In [ ]:
assert (ProcessRequest(Drink("water")) == "Water is healthy", "water")

assert (ProcessRequest(Drink("soda")) == "Soda is refreshing", "soda")

assert (ProcessRequest(Burger(true)) == "Mmm... cheeseburger", "cheeseburger")

assert (ProcessRequest(Burger(false)) == "Mmm... plain hamburger", "hamburger")

assert (ProcessRequest(Fries) == "Fries were a good decision", "fries")

assert (ProcessRequest(Drink("special request")) == "Couldn't match Drink(special request)", "other")

## Scala `trait`
* More flexible than abstract class in most ways
    * Can inherit from multiple traits
    * Can't take constructor parameters -- ex: `(variety: String)` from the Drink class above.
* Sometimes refered to as mixin (you'll see this language when you start reading Chipyard documentation)
    * Good conceptual model: think of inheriting from trait to "mix in" (think: mixing in ingredients during baking a cake) some needed functionality (or interface)
* Great in Chisel for adding a little functionality to different types of modules

Here's an example:

In [ ]:
trait PrintInSim {
    val printEnable = IO(Input(Bool()))
    
    def msg: String

    when (printEnable) {
        printf(p"$msg\n")
    }
}

class CounterMod extends Module with PrintInSim {
    val out = IO(Output(UInt(8.W)))
    def msg = "hello from counter"
    val count = Counter(255)
    out := count.value
}

test(new CounterMod) { c =>
    c.printEnable.poke(false.B)
    c.clock.step(2)
    c.printEnable.poke(true.B)
    c.clock.step(1)
    c.clock.step(1)
    c.clock.step(1)
    c.printEnable.poke(false.B)
    c.clock.step(1)
}

### Using Scala `traits` with Chisel Hardware
> Fill in the missing fields in the `DbgCounter` trait. The purpose of this trait is to add a debug counter that prints out the current cycle if the `debug` input (Bool) is true, otherwise count should remain at `0.U`. The trait should add an output `count` (UInt) representing the debug counter. _Hint:_ peek at the test case to infer signal names and data types.

In [ ]:
trait DbgCounter extends Module {
    // YOUR CODE HERE
    ???
}



In [ ]:
class TestMod(maxCycles: Int) extends Module with DbgCounter {
   def n = maxCycles 
}

def testDbgCounter(n: Int, printDbg: Boolean): Boolean = {
    test(new TestMod(n)) { dut =>
        dut.debug.poke(printDbg.B)
        if (printDbg) {
            for (i <- 0 until n) { 
                dut.count.expect(i.U)
                dut.clock.step()
            }
        }
        // test that counter wraps
        dut.count.expect(0.U)
    }
    true
}

assert(testDbgCounter(n=5, true))
assert(testDbgCounter(n=15, true))
assert(testDbgCounter(n=20, false))

## Scala Class Mechanism Recap (& Common Uses in Chisel)
* `class` - "regular" class, most commonly used
    * With Chisel, use for most things including modules, bundles, etc...
* `object` - singleton object, can be companion object
    * Can get multiple constructors (via factory method)
    * Can also group stateless code blocks or constants
* `case class` - restricted form of class with some functionality built-in
    * With Chisel, great for parameters and for use with pattern matching
* `abstract class` - virtual class useful when inherited
    * With Chisel, enables sharing functionality across different classes
* `trait` - like an interface in other languages, allows multiple inheritance
    * With Chisel, useful for "mixing-in" functionality
    * Not as rigid as inheriting from an abstract class

## Chisel Type Hierarchy
![](./images/type_hierarchy.svg)

[Source](https://github.com/chipsalliance/chisel?tab=readme-ov-file#data-types-overview)

## More Resources
* https://github.com/agile-hw/lectures/tree/main
* https://github.com/agile-hw - Original labs that these exercises were adapted from
* https://github.com/freechipsproject/chisel-bootcamp - Original Chisel Bootcamp Series (Some of the code here no longer functions due to lack of upkeep, but it can be a reference)
* https://github.com/Intensivate/learning-journey/wiki - Intensivate's Chisel Learning Journey